In [5]:
import cv2
import numpy as np

def homomorphic_filter(img, gamma_l=0.5, gamma_h=2.0, c=1, d0=10):
    # 转换成自然对数域
    img = np.log1p(np.array(img, dtype="float") / 255)
    
    # 中心化
    M, N = img.shape
    center_i, center_j = int(M/2), int(N/2)
    for i in range(M):
        for j in range(N):
            img[i, j] *= (-1) ** (i+j)
            
    # 频域变换
    img_fft = np.fft.fft2(img)
    
    # 构建同态滤波器函数H
    H = np.zeros_like(img_fft)
    for i in range(M):
        for j in range(N):
            H[i, j] = (gamma_h - gamma_l) * (1 - np.exp(-c * ((i-center_i)**2 + (j-center_j)**2) / (d0**2))) + gamma_l
    
    # 滤波
    filtered_img = np.real(np.fft.ifft2(img_fft * H))
    
    # 反中心化
    for i in range(M):
        for j in range(N):
            filtered_img[i, j] *= (-1) ** (i+j)
    
    # 反变换回空间域
    filtered_img = np.exp(np.clip(filtered_img, 0, None)) - 1
    filtered_img = np.uint8(filtered_img / np.max(filtered_img) * 255)
    
    return filtered_img

# 读取灰度图像
img = cv2.imread("./HF.jpg", 0)

# 对图像进行同态滤波
filtered_img = homomorphic_filter(img)

# 显示滤波结果
cv2.imshow("filtered image", filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
